# Exploratory Intent Discovery through Clustering

## What This Does

I started by trying SMAT (Stanford Mentor Alignment Tool) categories, but they were too abstract and didn't match real responses. So I turned to clustering to see what natural groupings actually appeared in the data. No predefined categories — just let the data tell me what's there.

## Why Start with Clustering?

After SMAT categories failed, I needed answers to:
- What intent categories actually exist in the real data?
- How do texts naturally group?
- Do intents overlap or are they separate?

**The big finding:** Clustering showed massive overlap. Many texts belong to multiple semantic groups. The discovered categories (12 natural ones from 96 initial clusters) were way more aligned with real-world responses than SMAT's theoretical framework. That's why I went with multi-label classification instead of forcing single labels.

## What I Did

1. Created multilingual sentence embeddings using sentence transformers
2. Used UMAP to reduce dimensions to 5D (makes clustering way faster)
3. Ran HDBSCAN clustering to find natural groups (leaf method finds smaller clusters too)
4. Merged 96 initial clusters down to ~10 categories that humans can actually understand

## What I Found

- Found 96 clusters initially — way more diversity than expected
- The biggest merged cluster had ~92% of texts. That's a red flag for single-label classification.
- Boundaries are fuzzy — lots of texts could belong to multiple categories
- This convinced me multi-label was necessary

See `decision_log.md` for more details on why I made these choices.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/merged_conversations_with_translations (1).csv')

/tmp/ipython-input-1956508705.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/merged_conversations_with_translations (1).csv')


In [ ]:
df.shape

(203578, 77)

In [ ]:
df_clean = df['person1_before_we_start'].dropna().drop_duplicates().sample(6000)
texts = df_clean.tolist()


In [ ]:
texts

['Data data tentang usaha yang sebenarnya.',
 'Get your facts true and ready',
 '1. Memiliki dasar keimanan yang kuat   \r\n2. mempunyai visi dan misi dalam menjalankan dan mengembangkan usaha \r\n3. mempunyai jiwa wirausaha',
 '1. Think about what you want to achieve in each session.\r\n2. Come prepared with no more than 3 big questions written down for our meetings.\r\n3. Be open and prepared for an interactive sessions. \r\n4. Expect homework/follow up items.',
 'Menyiapkan permasalahan yang dihadapi dan hal - hal apa yang paling penting untuk di beri pendampingan',
 'tener toda la información posible, para lograr una mentoria efectiva.',
 'Niat, doa, usaha dan konkrit dalam menjalankan komitmen',
 'Teman-teman adalah pemula yang baru/akan terjun ke dunia ilustrasi digital yang ingin belajar seputar skill dasar ilustrasi dan bagaimana cara mengkomersilkan sebuah karya. Teman-teman diharapkan sudah mengunduh aplikasi Ibis Paint dan Instagram yang Akan dipakai selama mentoring.',
 "We

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(texts, batch_size=32, show_progress_bar=True,convert_to_numpy=True)


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
from sklearn.preprocessing import normalize
emb_norm = normalize(embeddings)


In [ ]:
# Install if needed
!pip install umap-learn hdbscan --quiet

import umap
import hdbscan
from collections import Counter

# UMAP reduce to 5D with local neighborhood
reducer = umap.UMAP(n_neighbors=5, min_dist=0.0, n_components=5, random_state=42)
emb_umap = reducer.fit_transform(emb_norm)

# HDBSCAN: leaf selection, small clusters allowed
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=1, cluster_selection_method='leaf', metric='euclidean')
labels = clusterer.fit_predict(emb_umap)
print("Label counts:", Counter(labels))



/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Label counts: Counter({np.int64(-1): 214, np.int64(12): 21, np.int64(48): 20, np.int64(50): 16, np.int64(90): 16, np.int64(53): 15, np.int64(60): 15, np.int64(26): 15, np.int64(81): 14, np.int64(72): 13, np.int64(32): 12, np.int64(24): 12, np.int64(58): 12, np.int64(82): 12, np.int64(1): 11, np.int64(40): 11, np.int64(33): 11, np.int64(4): 11, np.int64(75): 11, np.int64(19): 11, np.int64(83): 11, np.int64(92): 11, np.int64(37): 11, np.int64(51): 10, np.int64(30): 10, np.int64(71): 10, np.int64(44): 10, np.int64(35): 10, np.int64(15): 10, np.int64(63): 10, np.int64(21): 10, np.int64(69): 9, np.int64(78): 9, np.int64(55): 9, np.int64(41): 9, np.int64(27): 9, np.int64(36): 8, np.int64(84): 8, np.int64(62): 8, np.int64(77): 8, np.int64(0): 8, np.int64(42): 8, np.int64(22): 8, np.int64(67): 8, np.int64(5): 8, np.int64(87): 8, np.int64(80): 7, np.int64(56): 7, np.int64(14): 7, np.int64(49): 7, np.int64(31): 7, np.int64(2): 7, np.int64(61): 7, np.int64(59): 7, np.int64(88): 7, np.int64(28): 7

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np

# Explicitly check lengths and align them to prevent ValueError
print(f"Length of texts: {len(texts)}")
print(f"Length of labels: {len(labels)}")

if len(texts) != len(labels):
    min_len = min(len(texts), len(labels))
    texts = texts[:min_len]
    labels = labels[:min_len]
    print(f"Adjusted texts length to: {len(texts)}")
    print(f"Adjusted labels length to: {len(labels)}")

df_out = pd.DataFrame({'text': texts, 'cluster': labels})

for c in sorted(df_out['cluster'].unique()):
    print(f"\n----- Cluster {c} -----")
    # Ensure the cluster is not empty before attempting to get examples
    if not df_out[df_out['cluster'] == c].empty:
        print(df_out[df_out['cluster'] == c]['text'].head(10).tolist())
    else:
        print("No texts found for this cluster.")


Length of texts: 1000
Length of labels: 1000

----- Cluster -1 -----
['Teman-teman adalah pemula yang baru/akan terjun ke dunia ilustrasi digital yang ingin belajar seputar skill dasar ilustrasi dan bagaimana cara mengkomersilkan sebuah karya. Teman-teman diharapkan sudah mengunduh aplikasi Ibis Paint dan Instagram yang Akan dipakai selama mentoring.', 'Prepare to work and commit to carry out every task for maximum results', 'Tener clara su necesidad de mentoria, para apoyarlo de forma efectiva', 'Bersiap untuk menjalankan usahanya step by step', 'Tener toda la documentación nformacion del negocio y su plan de crecimiento y tipos de productos', 'Be prepared with your questions and  to disclose / discuss business ideas/plans/goals  etc.', 'Let me know your expectations, what you sought to achieve.', 'ما الذي يأمل رائد الأعمال في تحقيقه من خلال عملية الإرشاد؟\r\nما هي التحديات التي يواجهها؟\r\nما هي المهارات أو المعرفة التي يبحث عن تطويرها؟', 'Menyiapkan pertanyaan terkait keluhan dan pr

In [ ]:
# === Merge many clusters into fewer human-friendly categories ===
# Run this after you have: texts (list), embeddings (np.array), labels (np.array or list)
# Adjust target_k to the number of final categories you want (e.g., 8, 10, 12)

import numpy as np
from collections import Counter, defaultdict
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer

def merge_clusters_to_k(texts, embeddings, labels, target_k=10, method='agglomerative'):
    texts = np.array(texts)
    labels = np.array(labels)
    uniq = sorted([c for c in np.unique(labels) if c != -1])  # skip -1 (optional)
    # if -1 present and many, treat -1 as its own clusters set; here we include -1 too
    if -1 in np.unique(labels):
        uniq = [-1] + [c for c in uniq if c != -1]
    # 1) compute centroid for each existing cluster
    centroids = []
    centroid_cluster_ids = []
    for c in uniq:
        idx = np.where(labels == c)[0]
        if len(idx) == 0:
            continue
        cent = embeddings[idx].mean(axis=0)
        centroids.append(cent)
        centroid_cluster_ids.append(c)
    centroids = np.vstack(centroids)  # shape (n_clusters_old, d)
    # 2) cluster the centroids to target_k groups
    if method == 'kmeans':
        km = KMeans(n_clusters=target_k, random_state=42, n_init=10).fit(centroids)
        merged_centroid_labels = km.labels_
    else:
        # agglomerative on centroid distances (works well for small #centroids)
        agg = AgglomerativeClustering(n_clusters=target_k, metric='cosine', linkage='average')
        merged_centroid_labels = agg.fit_predict(centroids)
    # map old cluster-id -> merged id
    old_to_merged = {old: int(merged_centroid_labels[i]) for i, old in enumerate(centroid_cluster_ids)}
    # 3) create final labels for each text (map its original label -> merged)
    merged_labels = []
    for lab in labels:
        # if label wasn't in old_to_merged (shouldn't happen) map to -1 else mapped label
        merged_labels.append(old_to_merged.get(int(lab), -1))
    merged_labels = np.array(merged_labels, dtype=int)
    # 4) get representative texts and top tf-idf terms per merged cluster
    df_by_cluster = defaultdict(list)
    for i, ml in enumerate(merged_labels):
        df_by_cluster[ml].append(i)
    # compute tfidf on full texts (lowercase; customize stop_words if you want)
    tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
    X = tfidf.fit_transform(texts.astype(str))
    terms = np.array(tfidf.get_feature_names_out())
    merged_info = {}
    for m in sorted(df_by_cluster.keys()):
        idxs = df_by_cluster[m]
        size = len(idxs)
        # representative texts (top by cluster size order)
        reps = [texts[i] for i in idxs[:7]]
        # compute top tfidf terms for this merged cluster (sum tfidf rows)
        cluster_tfidf = np.asarray(X[idxs].sum(axis=0)).ravel()
        top_idx = cluster_tfidf.argsort()[-12:][::-1]
        top_terms = terms[top_idx].tolist()
        merged_info[m] = {'size': size, 'examples': reps, 'top_terms': top_terms}
    return merged_labels, merged_info, old_to_merged

# Choose desired final category count here:
target_k = 10  # <--- change to 8,9,12, etc. until you like the granularity
merged_labels, merged_info, oldmap = merge_clusters_to_k(texts, embeddings, labels, target_k=target_k, method='agglomerative')

# Print summary
from collections import Counter
print("Original clusters:", len(set(labels)))
print("Merged to target_k =", target_k)
print("Merged label counts:", Counter(merged_labels))

# Print short summary of each merged cluster for naming
for m in sorted(merged_info.keys()):
    info = merged_info[m]
    print("\n=== Merged cluster", m, "size:", info['size'], "===")
    print("Top terms:", info['top_terms'][:10])
    print("Examples:")
    for ex in info['examples'][:5]:
        print("-", ex.replace("\n"," ")[:250])


Original clusters: 96
Merged to target_k = 10
Merged label counts: Counter({np.int64(0): 918, np.int64(1): 18, np.int64(2): 16, np.int64(4): 14, np.int64(8): 8, np.int64(3): 6, np.int64(9): 5, np.int64(7): 5, np.int64(6): 5, np.int64(5): 5})

=== Merged cluster 0 size: 918 ===
Top terms: ['business', 'dan', 'yang', 'tener', 'que', 'open', 'ready', 'clear', 'la', 'untuk']
Examples:
- Get your facts true and ready
 3. mempunyai jiwa wirausaha
 4. Expect homework/follow up items.
- Menyiapkan permasalahan yang dihadapi dan hal - hal apa yang paling penting untuk di beri pendampingan
- tener toda la información posible, para lograr una mentoria efectiva.

=== Merged cluster 1 size: 18 ===
Top terms: ['internet', 'media', 'dan', 'handphone', 'gadget', 'siapkan', 'social', 'pada', 'teknologi', 'platform']
Examples:
- Minimal gadget, internet, dan kemauan.
- Internet  data and little cost
- معرفة مشاكل المواقع ورائدي الاعمال
- - Teknologi dan Internet
- siapkan profil pada platform media sosi